In [1]:
import os
import time
import sys
import json
import numpy as np
import ipyvolume as ipv
from pathlib import Path
import numpy as np
import random
from datetime import datetime

from tyssue import Sheet, config
from tyssue.io import hdf5
from tyssue.generation import ellipsoid_sheet
from tyssue.draw.ipv_draw import view_ipv
from tyssue.draw.plt_draw import quick_edge_draw
from tyssue.dynamics import effectors, units
from tyssue.dynamics.sheet_gradients import height_grad
from tyssue.dynamics.factory import model_factory
from tyssue.solvers.sheet_vertex_solver import Solver
from tyssue.behaviors.events import EventManager
from tyssue.topology.sheet_topology import remove_face
from tyssue.utils.decorators import do_undo
from tyssue.utils import to_nd
from tyssue.io import hdf5


from invagination.ellipsoid import EllipsoidBGeometry as geom
from invagination.ellipsoid import VitellineElasticity
from invagination.delamination import (define_ovoid_mesoderm,
                                       initiate_monitoring_process,
                                       check_enter_in_process,
                                       check_tri_faces,
                                       type1_transition)

from invagination.basic_plot import mesoderm_position


import matplotlib.pyplot as plt

%matplotlib inline

SIM_DIR = Path('/home/admin-suz/Documents/Simulations')


In [2]:
import datetime

today = datetime.date.today()

#sim_save_dir = SIM_DIR/f'{today.isoformat()}_duree_traction'
sim_save_dir = SIM_DIR/f'2018-06-04_duree_traction'
try:
    os.mkdir(sim_save_dir)
except FileExistsError:
    pass

In [3]:
#%pdb

In [4]:
class RadialTension(effectors.AbstractEffector):
    
    dimensions = units.line_tension
    magnitude = 'radial_tension'
    label = 'Apical basal tension'
    element = 'vert'
    specs = {'vert':{'is_active',
             'height',
             'radial_tension'}}

    @staticmethod
    def energy(eptm):
        return eptm.vert_df.eval(
            'height * radial_tension * is_active')
         
    @staticmethod
    def gradient(eptm):
        grad = height_grad(eptm) * to_nd(
            eptm.vert_df.eval('radial_tension'), 3)
        grad.columns = ['g'+c for c in eptm.coords]
        return grad, None
    
    
EllipsoidSModel = model_factory(
    [
    RadialTension,
    VitellineElasticity,
    effectors.LineTension,
    effectors.FaceContractility,
    effectors.FaceVolumeElasticity,
    ], effectors.FaceAreaElasticity)


EllipsoidBModel = model_factory(
    [
    RadialTension,
    VitellineElasticity,
    #effectors.LineTension,
    effectors.FaceContractility,
    effectors.FaceAreaElasticity,
    effectors.CellVolumeElasticity,
    ], effectors.FaceAreaElasticity)


model = EllipsoidBModel
geom = EllipsoidBGeometry

In [5]:
dsets = hdf5.load_datasets('../data/hf5/ellipsoid_sheet_init.hf5',
                           data_names=['vert', 'edge', 'face', 'cell'])


with open('../data/json/ellipsoid.json', 'r+') as fp:
    specs = json.load(fp)

sheet = Sheet('ellipse', dsets, specs)

# Modify some initial value
sheet.settings['threshold_length'] = 1e-3
sheet.settings['vitelline_space'] = 0.2
sheet.vert_df['radial_tension'] = 0.
sheet.cell_df['prefered_vol'] = 4539601.384437251
sheet.cell_df['vol_elasticity'] = 3.e-6

fig, ax = quick_edge_draw(sheet, coords=list('zx'))

In [6]:
solver_kw = {'minimize': {'method': 'L-BFGS-B',
                          'options': {'ftol': 1e-8,
                                      'gtol': 1e-8}}}

## Define cells in the mesoderm

In [7]:
# Define rectangular mesoderm
#define_cell_in_mesoderm(sheet, 0.3, 0.7 ,0.05, 0.95)

# Define ovoid mesoderm
define_ovoid_mesoderm(sheet, 0, 0, 145, 40)

mesoderm = sheet.face_df[sheet.face_df.is_mesoderm].index
delaminating_cells = sheet.face_df[sheet.face_df['is_mesoderm']].index



print('number of apoptotic cells: {}'.format(delaminating_cells.size))
mesoderm_position(sheet, delaminating_cells)

In [9]:
from tyssue.behaviors.events import EventManager
from tyssue.topology.sheet_topology import remove_face
from tyssue.utils.decorators import do_undo
import os


import logging
logger = logging.Logger('event_log')

from tyssue.io import hdf5


@do_undo
def run_sim(sheet, mesoderm, geom, model, dirname, largeur_gauss, density_proba):
    solver = Solver
    
    # logger initiation
    event_logfile = os.path.join(dirname, 'events.log')
    hdlr = logging.FileHandler(event_logfile)
    hdlr.setLevel('INFO')
    logger.addHandler(hdlr)
    
    # initiate df for monitorign delamination process
    df = initiate_monitoring_process(sheet, mesoderm)
    list_cell_mesoderm = list(mesoderm)
    cell_in_delamination_process = []
    
    #Initiate manager
    manager = EventManager('face')

    
    t=0
    stop = 500
    while manager.current and t < stop:
        # Clean radial tension on all vertices
        sheet.vert_df['radial_tension'] = 0
        manager.execute(sheet)
        res = solver.find_energy_min(sheet, geom, model, **solver_kw)
        
        # add noise on vertex position to avoid local minimal.
        sheet.vert_df[['x', 'y']] += np.random.normal(scale=1e-3, size=(sheet.Nv, 2))
        geom.update_all(sheet)

        figname = os.path.join(
            dirname, 'invagination_{:04d}.png'.format(t))
        hdfname = figname[:-3]+'hf5'
        hdf5.save_datasets(hdfname, sheet)      
        
        # Test if face are done their transition or not (= if they are pulling)
        for f in cell_in_delamination_process : 
            face = sheet.idx_lookup(f, 'face')
            if face is not None : 
                if df.loc[f, 'transition'] == 0 :
                    if sheet.face_df.loc[face, 'area'] < sheet.settings['delamination']['critical_area'] : 
                        df.loc[f, 'transition'] = t            
        
        # Add cells in delamination process if they are authorized
        check_enter_in_process(sheet, manager, list_cell_mesoderm, 
                                    cell_in_delamination_process, df, t, 
                                    largeur_gauss, density_proba)
            
        # Check if cell is still in delamination process
        # if not put time in end column.
        temp_queu = manager.next.copy()
        for f in cell_in_delamination_process:
            present = False
            for tuple in temp_queu :
                if f in tuple : 
                    present = True
                    break                    
            if present == False : 
                df.loc[f,'end'] = t
                cell_in_delamination_process.remove(f)
        
        # Add cells with initially 3 neighbourghs to be eliminated
        check_tri_faces(sheet, manager)
        # Add T1 transition for face with at least one edge shorter than critical length
        [manager.append(type1_transition, f, kwargs=sheet.settings['T1']) for f in sheet.edge_df[
            sheet.edge_df['length'] < sheet.settings['T1']['critical_length']]['face'].unique()]
    
    
        manager.update()
        df.to_csv (os.path.join(dirname,'time_position.csv'))
        t+=1
        
        
    logger.removeHandler(hdlr)
    return sheet


In [10]:
def delamination_process(sheet, contractility_rate, critical_area, 
                         radial_tension, largeur_gauss, densite_proba, nb_iteraction_max):

    # Directory definition 
    dirname = '{}_contractility_{}_critical_area_{}_radialtension_{}_nb_iteraction_max'.format(
                contractility_rate, critical_area, radial_tension, nb_iteraction_max)
    dirname = os.path.join(sim_save_dir, dirname)
    
    print('starting {}'.format(dirname))
    try:
        os.mkdir(dirname)
    except IOError:
        pass
    
    settings = {'contract_rate': contractility_rate,
            'critical_area': critical_area,
            'radial_tension': radial_tension,
            'nb_iteration':0,
            'nb_iteration_max':nb_iteraction_max,
            'contract_neighbors':True,
            'critical_area_neighbors':10,
            'geom': geom}
    
    solver = Solver
    
    # Add some information to the sheet
    sheet2 = sheet.copy(deep_copy=True)
    
    sheet2.face_df['id'] = sheet2.face_df.index.values
    sheet2.settings['delamination'] = settings
    
    settings2 = {'critical_length':0.3}
    sheet2.settings['T1'] = settings2
    #""" Initiale find minimal energy
    # To be sure we are at the equilibrium
    res = solver.find_energy_min(sheet2, geom, model, **solver_kw)
   
    sheet2 = run_sim(sheet2, delaminating_cells, geom, model, dirname, largeur_gauss, densite_proba)

    print('{} done'.format(dirname))
    print('~~~~~~~~~~~~~~~~~~~~~\n')
    

In [12]:
from datetime import datetime

global_start=datetime.now()
print ("start : " + str(global_start))

radial_tension = [50]
contractility_percent = [8]
contractility_rate = [1+c/100 for c in contractility_percent]
lg = 17
dp = 2
#nb_iter = [5, 10, 20, 30, 40, 50]
nb_iter =[ 5, 10, 30]
for rad in radial_tension:
    for contracts in contractility_rate:
        for nb_iteraction_max in nb_iter : 
            print ('\tnb_iter : ' + str(nb_iteraction_max))
            delamination_process(sheet, contractility_rate[0], 5, radial_tension[0], lg, dp, nb_iteraction_max)

            
global_end = datetime.now()
print ("end : " + str(global_end))
print ('Duree totale d execution : \n\t\t')
print (global_end-global_start)

In [11]:
# Multi-execution parallel 
from joblib import Parallel, delayed
import multiprocessing
from datetime import datetime

global_start=datetime.now()
print ("start : " + str(global_start))
num_cores = multiprocessing.cpu_count()


radial_tension = [50]
contractility_percent = [8]
contractility_rate = [1+c/100 for c in contractility_percent]
lg = 17
dp = 2


nb_iter = [5, 10, 20, 30, 40, 50]
nb_iterations = [30, 10, 5]


results = Parallel(n_jobs=2)(delayed(delamination_process)(
    sheet, contractility_rate[0], 5, radial_tension[0], lg, dp, nb_iteraction_max)
                             for nb_iteraction_max in nb_iterations)

global_end = datetime.now()
print ("end : " + str(global_end))
print ('Duree totale d execution : \n\t\t')
print (global_end-global_start)


In [25]:
for val in color_d.values():
    print (val)

In [14]:
SIM_DIR = Path('/media/admin-suz/Sophie/2018/Papiers-EMT-Melanie/datas/')

In [29]:
import matplotlib.colors as colors
import matplotlib.cm as cmx
import math
jet = plt.get_cmap('gist_rainbow')
cNorm  = colors.Normalize(vmin=0, vmax=5)
scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=jet)
dirname = SIM_DIR/'2018-06-04_duree_traction'
list_dir = os.listdir(dirname)

list_color = [(1.0, 0.0, 0.16, 1.0),
            (1.0, 0.5, 0.0, 1.0),
            (0.0, 0.5, 0.0, 1.0),
            (0.0, 0.5, 1.0, 1.0),
            (0.16304347826086973, 0.0, 1.0, 1.0),
            (1.0, 0.0, 0.75, 1.0)]


"""
dict_stop= { '1.08_contractility_5_critical_area_50_radialtension_5_nb_iteraction_max':25,
             '1.08_contractility_5_critical_area_50_radialtension_10_nb_iteraction_max':31,
             '1.08_contractility_5_critical_area_50_radialtension_20_nb_iteraction_max':36,
             '1.08_contractility_5_critical_area_50_radialtension_30_nb_iteraction_max':33,
             '1.08_contractility_5_critical_area_50_radialtension_40_nb_iteraction_max':35,
             '1.08_contractility_5_critical_area_50_radialtension_50_nb_iteraction_max':34}"""
list_dir = [ '1.08_contractility_5_critical_area_50_radialtension_5_nb_iteraction_max',
             '1.08_contractility_5_critical_area_50_radialtension_10_nb_iteraction_max',
             '1.08_contractility_5_critical_area_50_radialtension_20_nb_iteraction_max',
             '1.08_contractility_5_critical_area_50_radialtension_30_nb_iteraction_max',
             '1.08_contractility_5_critical_area_50_radialtension_40_nb_iteraction_max',
             '1.08_contractility_5_critical_area_50_radialtension_50_nb_iteraction_max']
color_d={}
i=0
for directory in list_dir: 
    color_d [directory] = list_color[i]
    i=i+1

fig = plt.figure()

fig, (ax) = plt.subplots(1,1)
for directory in list_dir:
    sheet = open_sheet(os.path.join(dirname, directory), 0)
    sheet_mesoderm = sheet.extract('is_mesoderm')

    subset_mesoderm_face_df = sheet_mesoderm.face_df[
        (sheet_mesoderm.face_df['z'] > -20)
        & (sheet_mesoderm.face_df['z'] < 20) &
        (sheet_mesoderm.face_df['x'] > -2)
        & (sheet_mesoderm.face_df['x'] < 2)]

    depth_0 = np.mean(subset_mesoderm_face_df[
        'rho'])
    
    depths=[]
    for t in range(0, 500):
        sheet = open_sheet(os.path.join(dirname, directory), t)
        sheet_mesoderm = sheet.extract('is_mesoderm')
        subset_mesoderm_face_df = sheet_mesoderm.face_df[
            (sheet_mesoderm.face_df['z'] > -20)
            & (sheet_mesoderm.face_df['z'] < 20) &
            (sheet_mesoderm.face_df['x'] > -2)
            & (sheet_mesoderm.face_df['x'] < 2)]

        depths.append(
            depth_0 - np.mean(subset_mesoderm_face_df['rho'] ))
    """print (directory)
    print (max(depths))
    print (depths.index(max(depths)))
    print ('\n')"""
    #ax.plot (np.arange(0,dict_stop[directory]), depths[0:dict_stop[directory]], label = str(directory).split('_')[7], color=color_d[directory])
    
    ax.plot (np.arange(0,len(depths)), depths, label = str(directory).split('_')[7], color=color_d[directory])
    

    ax.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0., prop={'size': 12, 'family':'Arial'})
ax.set_xlabel('Time step', size=12, family='sans-serif', fontname='Arial')
ax.set_ylabel('Depth (µm)', size=12, family='sans-serif', fontname='Arial')
for tick in ax.xaxis.get_ticklabels():
    tick.set_fontsize(12)
    tick.set_fontname('Arial')
for tick in ax.yaxis.get_ticklabels():
    tick.set_fontsize(12)
    tick.set_fontname('Arial')                                                                                 



fig.set_size_inches(6, 4.5)

fig.savefig('../test_duree_depth.svg', dpi = 150)